In [1]:
from pywebio import start_server
from pywebio.input import *
from pywebio.output import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc("font",family='Noto Sans CJK JP')

# 定义颜色字典
color_map = {
    '宫调': '#8B4513',  # 土褐色
    '商调': '#4682B4',  # 钢青色
    '角调': '#32CD32',  # 鲜绿色
    '徵调': '#FF4500',  # 橙红色
    '羽调': '#ADD8E6'   # 淡蓝色
}

# 设置全局图表大小
plt.rc('figure', figsize=(16, 12))
plt.rc('axes', titlesize=20)  # 设置轴标题的字体大小
plt.rc('axes', labelsize=16)  # 设置轴标签的字体大小
plt.rc('xtick', labelsize=14)  # 设置x轴刻度标签的字体大小
plt.rc('ytick', labelsize=14)  # 设置y轴刻度标签的字体大小
plt.rc('legend', fontsize=14)  # 设置图例的字体大小

REC_NUM = 10

df = pd.read_csv('./data/final_classify_data.csv')

# 时区
time_zone_count = df['time_zone'].value_counts()
time_zone_count_filter = time_zone_count[time_zone_count > 500]
time_zone_list = list(time_zone_count.index)

# 标签
hashtags = df['hashtag'].value_counts()
hashtags = hashtags[hashtags > 100][:30]
hashtags_list = list(hashtags.index)

wuyin_table = pd.read_excel('./data/五声音阶的特征.xlsx', sheet_name='Sheet1')
wuyin_table

styled_wuyin_table = """
<style>
    table {
        width: 100%;
        border-collapse: collapse;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
        font-size: 14px;
    }
    th {
        background-color: #f2f2f2;
    }
    .text-wrap {
        white-space: normal;
    }
</style>
""" + wuyin_table.to_html(index=False, classes='text-wrap')


In [2]:
import base64
import requests

# Spotify 开发者平台注册的应用程序的客户端 ID 和客户端秘钥
client_id = 'a4a6ca001e074c828ccd20cf9643cba5'
client_secret = 'a4162505cf484701b0ac2d3128c80b27'

# 构造认证请求的 URL
auth_url = 'https://accounts.spotify.com/api/token'

# 构造 HTTP 头部，包含客户端 ID 和客户端秘钥
headers = {
    'Authorization': 'Basic ' + base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode(),
}

# 构造请求参数
data = {
    'grant_type': 'client_credentials',
}

# 发送请求，获取访问令牌
response = requests.post(auth_url, headers=headers, data=data)

# 解析响应，提取访问令牌
token = response.json().get('access_token')

print(token)

# token = 'BQBMDms9U2jucLNhdOe47Zys1n3hNE5Cw2sWzAS3SOyXR3-KsrF0XDwvgD4Z_T1h0HBTMPi9_I1tiCR5M-9hs471ruZ9s8TaHmeC9FUNE8kfZfS5qqo'
# print(token)

BQC3qAePfYGC6dOujDQ47U4il9ClT_XJOW5NTwGHrypWaDRAoHM1kpV5F2MMslmFjMHHmHbSzOoYFRAuykgGNSiTGmH12f819vT_o4Ipzj72bcfyEM8


In [3]:
# 网络配置

local_ip = '100.78.231.77'
proxies = {
    "http": f"http://{local_ip}:7890",
    "https": f"http://{local_ip}:7890"
}

In [4]:
time_range = {
    '宫调': [(6, 9), (11, 15), (17, 19)],
    '商调':[(15, 17)],
    '角调':[(19, 23)],
    '徵调':[(21, 23)],
    '羽调':[(7, 11)]
}
mood_to_track_type = {
    '抑郁': '商调',
    '悲伤': '徵调',
    '平静': '角调',
    '开心': '宫调',
    '兴奋': '羽调'
}

In [5]:
# 标签映射，todo
label_mapping = {0: '宫调', 1: '商调', 2: '角调', 3: '徵调', 4: '羽调'}

# 使用 replace() 方法进行映射
df['labels'] = df['labels'].replace(label_mapping)

In [6]:
def get_time():
        # 获取当前时间戳
    timestamp = time.time()

    # 将时间戳转换为datetime对象
    current_datetime = datetime.fromtimestamp(timestamp)

    # 提取年、月、日、小时和星期几
    year = current_datetime.year
    month = current_datetime.month
    day = current_datetime.day
    hour = current_datetime.hour
    weekday = current_datetime.weekday()  # 注意：这里星期一是0，星期天是6

    return {'year':year, 'month': month, 'day':day, 'hour': hour, 'weekday':weekday + 1}

def classify_time_period(hour):
    if hour < 6:
        return 'Wee hours(00:00 - 6:00)'
    elif hour < 12:
        return 'Morning(6:00 - 12:00)'
    elif hour < 18:
        return 'Afternoon(12:00 - 18:00)'
    else:
        return 'Night(18:00 - 24:00)'

In [ ]:
from pywebio import config
from pywebio import pin
from pywebio.session import *
from functools import partial
import asyncio
import nest_asyncio
import io
import re
import seaborn as sns
import requests
import json
from datetime import datetime
import time

self_sample = None

def show_info(sample, index = False):
    # 通过PyWebIO输出HTML
    put_html(sample.to_html(index=index))
    
def show_plt():
    # 图片存入内存
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    
    put_image(buf.read())
    plt.close()

def get_mood(score):
    if score < 0.2:
        return '抑郁'
    elif score < 0.4:
        return '悲伤'
    elif score < 0.6:
        return '平静'
    elif score < 0.8:
        return '开心'
    else:
        return '兴奋'
    
def set_sample():
    score = pin.pin['score']
    time_zone = pin.pin['time_zone']
    hashtag = pin.pin['hashtag']
    lang = pin.pin['lang']
    t = get_time()
    sample = pd.DataFrame({
        'score' :score,
        'mood': get_mood(score),
        'hashtag':hashtag,
        'lang':lang,
        'tweet_lang':lang,
        'time_zone':time_zone,
        'month':t['month'],
        'hours':t['hour'],
        'season':t['month'] // 4,
        'day_time':classify_time_period(t['hour']),
        'weekday_index':t['weekday'],
        'weekend':(t['weekday'] > 5)
    }, index=[0])
    
    with use_scope('sample', clear = True):
        put_markdown('#### 您当前的样本为：')
        show_info(sample)
        
    global self_sample
    self_sample = sample

def play_music(music_url):
    audio_html = f"""
    <audio controls>
      <source src="{music_url}" type="audio/mpeg">
      Your browser does not support the audio element.
    </audio>
    """
    put_html(audio_html)
    
def get_time_zone_info():
    if self_sample is None:
        popup('Error', put_error('请您先确认个人信息！'))
        return
    # 时区人数图
    time_zone_count_filter.plot(kind='bar')
    time_zone = pin.pin['time_zone']
    
    plt.xlabel('Time Zone')
    plt.ylabel('Frequency')
    plt.title('Frequency of Time Zones')
    
    with use_scope('display_zone', clear = True):
        show_plt()
        
        put_text(f'您所在的时区{time_zone}，年度听歌人数为：{time_zone_count[time_zone]}。')
    
def get_time_zone_year():
    if self_sample is None:
        popup('Error', '请您先确认个人信息！')
        return
    # 1月份-12月份听歌人数的变化
    data_group_by_month = df.groupby(by=['month', 'time_zone']).size().unstack(fill_value=0).stack().reset_index(name='count')
    data_group_by_month = data_group_by_month[data_group_by_month['time_zone'].isin(time_zone_count_filter.index)]

    # 使用seaborn绘制折线图
    sns.lineplot(data=data_group_by_month, x='month', y='count', hue='time_zone', palette='bright')
    plt.title('Listeners by Time Zone Over Month')
    plt.ylabel('Number of Listeners')
    plt.xlabel('Month')
    plt.legend(loc='best')
    plt.xticks(rotation=45)
    
    with use_scope('display_zone', clear = True):
        show_plt()
        
def get_hashtag_info():
    if self_sample is None:
        popup('Error', '请您先确认个人信息！')
        return
    fig, axes = plt.subplots(1, 1)
    hashtags.plot.barh(ax=axes, color='#2ecc71', alpha=0.7) # 翠绿
    plt.xlabel('Frequency')
    
    hashtag = pin.pin['hashtag']
    with use_scope('display_zone', clear = True):
        show_plt()
        put_text(f'和您使用相同标签#{hashtag}的人数为：{hashtags[hashtag]}。')
        
def set_initial_info():
    pin.put_slider('score', label='请输入您目前的情绪分数(0 - 1，代表消极 - 积极)', min_value=0.0, max_value=1.0, step=0.1, value=0.5, required=True),
    pin.put_select('time_zone', label = '请选择您所在的时区', options=time_zone_list, value = 'Beijing'),
    pin.put_select('hashtag', label = '请选择一个推文标签', options=hashtags_list, value = 'np'),
    pin.put_radio('lang', label='您所在时区的语言是', options=['en', 'nl', 'zh-tw', 'zh-Hans'], value = 'zh-Hans')
    put_buttons(['确定', '显示时区听歌调式分布', '显示时区听歌人数', '显示时区年度听歌趋势', '显示标签热度'], 
                onclick=[set_sample, get_time_zone_label, get_time_zone_info, get_time_zone_year, get_hashtag_info])
    
def get_time_zone_label():
    if self_sample is None:
        popup('Error', '请您先确认个人信息！')
        return
    df_group = df.groupby(by = ['time_zone', 'labels']).size().reset_index(name='count')
    df_group_sample = df_group[df_group['time_zone'] == self_sample['time_zone'][0]]

    # 根据labels生成颜色列表
    colors = [color_map[label] for label in df_group_sample['labels']]

    # 绘制饼图
    plt.pie(df_group_sample['count'], labels=df_group_sample['labels'], autopct='%1.1f%%', startangle=140, shadow=True, colors=colors, textprops={'fontsize': 16})
    plt.title(f'{self_sample["time_zone"][0]} Tones Distribution', fontsize=20)  # 设置标题
    with use_scope('display_zone', clear = True):
        show_plt()
def get_tracks(track_ids):
    # Spotify API 请求的 URL
    url = 'https://api.spotify.com/v1/tracks'
    
    # 将 track ID 列表转换为逗号分隔的字符串，作为请求参数
    params = {
        'ids': ','.join(track_ids)
    }
    
    # 构造请求头，包含访问令牌
    headers = {
        'Authorization': f'Bearer {token}',
    }

    # 发送 GET 请求，获取 track 的信息
    response = requests.get(url, headers=headers, params=params)

    # 解析响应，获取 track 的信息
    print(response.text)
    tracks = response.json()
    tracks = tracks['tracks']
    
    return tracks

def get_playlist_html(tracks):
    html_code = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>My Music Playlist</title>
    <style>
    body {
        font-family: Arial, sans-serif;
        background-color: #f0f0f0;
        margin: 0;
        padding: 0;
    }

    .container {
        max-width: 800px;
        margin: 20px auto;
        padding: 20px;
        background-color: #fff;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    }

    .song {
        display: flex;
        align-items: center;
        justify-content: space-between;
        border-bottom: 1px solid #ddd;
        padding: 10px 0;
    }

    .song img {
        width: 120px;
        height: 120px;
        margin-right: 20px;
    }

    .song-details {
        flex-grow: 1;
    }

    .song-title {
        font-size: 1.2em;
        font-weight: bold;
        margin: 0 0 5px;
    }

    .song-artist {
        font-size: 1em;
        margin: 0 0 5px;
    }

    .song-album {
        font-size: 0.9em;
        color: #666;
        margin: 0 0 5px;
    }

    .song-release-date {
        font-size: 0.9em;
        color: #666;
        margin: 0 0 5px;
    }

    .play-button {
        padding: 5px 10px;
        background-color: #1db954;
        color: #fff;
        border: none;
        border-radius: 5px;
        cursor: pointer;
    }

    .play-button:hover {
        background-color: #1ed760;
    }
    </style>
    </head>
    <body>
    <div class="container">
    """
    
    for track in tracks:
        song_html = f"""
        <div class="song">
            <img src="{track['album']['images'][0]['url']}" alt="Album Image">
            <div class="song-details">
                <div class="song-title">{track['name']}</div>
                <div class="song-artist">artist: {track['artists'][0]['name']}</div>
                <div class="song-album">album: {track['album']['name']}</div>
                <div class="song-release-date">release date: {track['album']['release_date']}</div>
                <div class="song-url">{track['external_urls']['spotify']}</div>
            </div>
        </div>
        """
        html_code += song_html

    html_code += """
    </div>
    </body>
    </html>
    """
    
    return html_code
def get_preview_url(url):
    try:
        # 向指定的 URL 发送 GET 请求，通过代理进行
        response = requests.get(url ,proxies=proxies)
        
        # 定义正则表达式模式，这里是找了音乐片段的url，若要全音频，需要另想办法
        pattern = r'<meta property="og:audio" content="(https://[^"]+)"'
        
        # 使用正则表达式查找匹配的内容
        match = re.search(pattern, response.text)
        if match:
            audio_url = match.group(1)
            return audio_url
            
        if response.status_code == 200:
            print("Get_preview_url successful!")
        else:
            print("Failed to reach Spotify.")
    except requests.exceptions.RequestException as e:
        print("Error during requests to {0} : {1}".format(url, str(e)))
        
def zone_based_rec():
    # 处理数据，获得时区前10
    time_zone = self_sample['time_zone'][0]
    df_time_zone = df[df['time_zone'] == time_zone]
    group_df = df_time_zone.groupby(by=['track_id'])
    number_df = group_df.count()
    
    # 选择第一列
    first_column = number_df.iloc[:, 0]  # 保留第一列

    # 创建一个新的 DataFrame 并重命名列为 'count'
    new_df = pd.DataFrame(first_column).rename(columns={first_column.name: 'count'})

    # 按 'count' 列从大到小排序
    new_df = new_df.sort_values(by='count', ascending=False)

    # 指定要获取信息的 track ID 列表
    track_ids = new_df.index[:10]

    tracks = get_tracks(track_ids)
    
    # 打印 track 的信息
    print(tracks)
    print(type(tracks))
    
    html_code = get_playlist_html(tracks)

    playlist = [{
        'image_url': track['album']['images'][0]['url'], 
        'title' : track['name'],
        'artist': track['artists'][0]['name'],
        'audio_url': get_preview_url(track['external_urls']['spotify']),
        'lyric': ''
    } for track in tracks]
    
    with use_scope('display_zone', clear = True):
        put_markdown('---')
        put_markdown(f'**为您推荐了时区{time_zone}最热门的10首歌曲**')
        put_buttons(['Play'], onclick=[partial(music_player, playlist=playlist)])
        put_html(html_code)
    
    
def time_based_rec():
    hour = self_sample['hours'][0]
    
    rec_label = set()
    for label, times in time_range.items():
        for start, end in times:
            if hour >= start and hour < end:
                rec_label.add(label)
                
    with use_scope('display_zone', clear = True):
        if len(rec_label) == 0:
            put_markdown(f'**您的时间不在以上区间内，推测您在夜间听歌，为您推荐商、羽调歌曲帮助睡眠！**')
            rec_label.add('商调', '羽调')
        else:
            put_markdown(f'**您的时间为：{hour}时，适合听的歌曲调式为：{rec_label}**')

        rec_df = df[df['labels'].isin(rec_label)]
        rec_list = rec_df.sample(REC_NUM)[['track_id', 'labels']]
        show_info(rec_list)
    
def emotion_based_rec():
    score = self_sample['score'][0]
    mood = self_sample['mood'][0]
    track_type = mood_to_track_type[mood]
    
    rec_list = df[df['labels'] == track_type].sample(REC_NUM)[['track_id', 'labels']]
    with use_scope('display_zone', clear = True):
        put_markdown(f'**您输入的情绪分数为：{score}，根据您输入的信息情绪检测为：{mood}，为您推荐“{track_type}”歌曲！**')
        show_info(rec_list)
        

def total_rec():
    pass

def get_player_html(playlist):
    html_content = f"""
    <html>
    <head>
        <style>
            .player-container {{
                background-color: #f0f0f0;
                padding: 20px;
                width: 100%;
                max-width: 600px;
                margin: auto;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
                text-align: center;
                font-family: Arial, sans-serif;
            }}
            button {{
                margin: 5px;
                padding: 5px 20px;
                font-size: 16px;
                background-color: #4CAF50;
                color: white;
                border: none;
                border-radius: 5px;
                cursor: pointer;
            }}
            button:hover {{
                background-color: #45a049;
            }}
            #trackInfo {{
                margin-top: 10px;
                font-size: 18px;
                color: #333;
            }}
            .lyrics {{
                text-align: center;  /* 文字居中显示 */
                font-size: 20;     /* 文字大小 */
                line-height: 36px;   /* 行高 */
                margin-top: 10px;   /* 顶部间距 */
                white-space: pre-line; /* 换行 */
            }}
        </style>
    </head>
    <body>
        <div class="player-container">
            <img id="trackImage" src="{playlist[0]['image_url']}" style="width: 200px; height: 200px; border-radius: 10px; margin-bottom: 10px;">
            <div id="trackInfo">{playlist[0]['title']}</div>
            <audio id="audioPlayer" controls autoplay>
                <source src="{playlist[0]['audio_url']}" type="audio/mpeg">
                Your browser does not support the audio tag.
            </audio>
            <br>
            <button onclick="previousTrack()">Previous</button>
            <button onclick="nextTrack()">Next</button>
            <div class="lyrics", id="lyric">
                {playlist[0]['lyric']}
            </div>
            
        </div>
        <script>
            var playlist = {playlist};
            var currentTrack = 0;
            var player = document.getElementById('audioPlayer');
            var trackInfo = document.getElementById('trackInfo');
            var trackImage = document.getElementById('trackImage')
            var lyric = document.getElmentById('lyric')

            function setTrack(track) {{
                player.src = track['audio_url'];
                player.load();
                player.play();
                trackInfo.innerText = track['title'];
                trackImage.src = track['image_url'];
                lyric.innerText = track['lyric'];
            }}

            function nextTrack() {{
                if (currentTrack < playlist.length - 1) {{
                    currentTrack++;
                }} else {{
                    currentTrack = 0; // Loop back to the first track
                }}
                setTrack(playlist[currentTrack]);
            }}

            function previousTrack() {{
                if (currentTrack > 0) {{
                    currentTrack--;
                }} else {{
                    currentTrack = playlist.length - 1; // Loop to the last track
                }}
                setTrack(playlist[currentTrack]);
            }}
        </script>
    </body>
    </html>
    """
    return html_content

def music_player(playlist):
    # 将 Python 列表转换为 JavaScript 数组格式的字符串
    html_content = get_player_html(playlist)
    put_html(html_content)

def play_track():
    url = "https://p.scdn.co/mp3-preview/3ff56360de277b2cc32d713e0798d1ff63b8e8a8?cid=cfe923b2d660439caf2b557b21f31221"
    audio_html = f"""
    <audio controls>
        <source src="{url}" type="audio/mp3">
        Your browser does not support the audio element.
    </audio>
    
    """
    put_html(audio_html)

def page1():
    clear('display_zone')
    clear('page3')
    with use_scope('page_content', clear=True):  # 清除页面内容作用域中的旧内容
        set_initial_info()

def page2():
    if self_sample is None:
        popup('Error', '请您先确认个人信息！')
        return
    clear('page_content')
    clear('display_zone')
    clear('page3')
    
    with use_scope('page_content', clear=True):  # 清除页面内容作用域中的旧内容
        put_html(styled_wuyin_table)
        put_text('表格整理自')
        put_link('1、网易文章', 'https://ds.163.com/article/5faa98d2f7feba4623704eae/')
        put_link(' 2、论文《宫商角徵羽，音乐养生趣味多》', 'https://kns.cnki.net/kcms2/article/abstract?v=29axctaKF3zEV--IRp7m2uQIKb9z2zY76Yu3JRRy7OprmsI-LS0sP_h0ykdBtF_CYbvqe0CkuZEg6yD04P3-1Rfliy9vFo-ygRt47aunFCYDxBirmwWeIv22prGVAX0K7Z9AqXYdptY=&uniplatform=NZKPT&flag=copy')
        put_markdown('---')
        put_buttons(['推荐时区最热门top10', '基于时间推荐', '基于情绪推荐', '综合推荐'], 
                onclick=[zone_based_rec, time_based_rec, emotion_based_rec, total_rec])
def page3():
    if self_sample is None:
        popup('Error', '请您先确认个人信息！')
        return
    clear('page_content')
    clear('display_zone')
    clear('sample')
    
    BASE_URL = 'http://10.80.43.30:3000/api/'
    GEN_API = 'generate'
    
    info = input_group("Music Gen",[
      textarea('Please describe your mood within 500 words, the more description, the richer the content!', rows = 10, name='prompt'),
      radio('Pure music or with lyric', options=[('Pure music', True), ('With lyric', False)], required=True, name='instrumental')
    ])
    
    prompt = info['prompt']
    instrumental = info['instrumental']
    
    print(prompt, '\n', instrumental)
        
    # 调用suno AI 的 API 生成音乐
    request_body = {
      "prompt": prompt,
      "tags": self_sample['hashtag'][0],
      "make_instrumental": instrumental,
      "wait_audio": True
    }
    
     # 显示加载标识，来自（output模块）
    with put_loading():
        put_text("Generating music. Please waiting for a minute......")
        response = requests.post(BASE_URL + GEN_API, json = request_body)

    # 打印响应体（内容）
    track_list = json.loads(response.text)
    print(track_list)

    with use_scope('page3', clear=True): 
        put_success('Successful generating. Recommendate two tracks for you. Enjoying it!', closable=True)
        music_player(playlist=track_list)
        
def navigation_bar():
    with use_scope('navbar', clear=True):  # 使用独立的作用域，并在更新时清除旧内容
        put_buttons(['Basic Info', 'Music Rec', 'Music Gen'], onclick=[page1, page2, page3])
        put_markdown('---')
        
@config(theme="minty")
def main():
    """Emotion-based Recommendation"""
    
    navigation_bar()  # 初始时设置导航条
    page1()  # 默认显示第一页内容
        

if __name__ == '__main__':
    nest_asyncio.apply()
    asyncio.create_task(start_server(main, port=9999, debug=True, notebook=True))
    

Running on all addresses.
Use http://10.80.43.30:9999/ to access the application
{
  "tracks" : [ {
    "album" : {
      "album_type" : "compilation",
      "artists" : [ {
        "external_urls" : {
          "spotify" : "https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of"
        },
        "href" : "https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of",
        "id" : "0LyfQWJT6nXafLPZqxe9Of",
        "name" : "Various Artists",
        "type" : "artist",
        "uri" : "spotify:artist:0LyfQWJT6nXafLPZqxe9Of"
      } ],
      "available_markets" : [ "AE", "AL", "AR", "AT", "AU", "BA", "BE", "BG", "BH", "BO", "BR", "CA", "CH", "CI", "CL", "CM", "CO", "CR", "CW", "CY", "CZ", "DE", "DK", "DO", "DZ", "EC", "EE", "EG", "ES", "FI", "FR", "GB", "GH", "GR", "GT", "HK", "HN", "HR", "HU", "ID", "IE", "IL", "IN", "IS", "IT", "JP", "KE", "KH", "KR", "KW", "LB", "LK", "LT", "LU", "LV", "MA", "ME", "MK", "MT", "MX", "MY", "NG", "NI", "NL", "NO", "NZ", "OM", "PA", "PE", "PH", "PL", "PT

宫调：浑和厚重如土，旋律悠扬
颜色：#8B4513 (土褐色)
理由：土褐色象征大地和稳重，与宫调的厚重和悠扬特性相匹配。
商调：初秋清凉、悲伤
颜色：#4682B4 (钢青色)
理由：钢青色给人一种清新凉爽的感觉，适合表达商调的清凉和悲伤情感。
角调：清脆如木，曲调亲切
颜色：#32CD32 (鲜绿色)
理由：鲜绿色代表新生和活力，象征木的生命力和亲切感，与角调的特点吻合。
徵调：热烈如火，节奏欢快
颜色：#FF4500 (橙红色)
理由：橙红色热情且引人注目，能很好地代表徵调的热烈和欢快。
羽调：苍凉淡荡如水，风格清纯
颜色：#ADD8E6 (淡蓝色)
理由：淡蓝色给人一种宁静和清纯的感觉，适合羽调的苍凉和清纯特性。

In [ ]:
from pywebio.platform.flask import webio_view
from pywebio import start_server
from pywebio.output import put_html, put_button
from flask import Flask
import asyncio
import nest_asyncio

def main():
    # HTML 和 JavaScript 代码片段
    html_content = """
    <h1>Spotify Web Playback SDK Example</h1>
    <div id="player"></div>
    <script src="https://sdk.scdn.co/spotify-player.js"></script>
    <script>
    window.onSpotifyWebPlaybackSDKReady = () => {
        const token = 'BQCLezhq0iVaYqFkLcYNrRtJdUsWkN-mFpZxveiSVapifE_oa5aslQ8TPSd8Sw-rXxoryKtmFOP2_xJf2gAJ6ISg7NrePcmn-yNJF3Dhd7mGHalax5A'; // 在这里填写你的 Spotify 访问令牌
        const player = new Spotify.Player({
            name: 'Your Awesome Web Player',
            getOAuthToken: cb => { cb(token); }
        });

        // 错误处理
        player.addListener('initialization_error', ({ message }) => { console.error(message); });
        player.addListener('authentication_error', ({ message }) => { console.error(message); });
        player.addListener('account_error', ({ message }) => { console.error(message); });
        player.addListener('playback_error', ({ message }) => { console.error(message); });

        // Playback status updates
        player.addListener('player_state_changed', state => { console.log(state); });

        // Ready
        player.addListener('ready', ({ device_id }) => {
            console.log('Ready with Device ID', device_id);
        });

        // Connect to the player!
        player.connect();
    };
    </script>
    """

    put_html(html_content)  # 将 HTML 和 JavaScript 代码放到网页上

if __name__ == '__main__':
    nest_asyncio.apply()
    asyncio.create_task(start_server(main, port=8080, debug=True, notebook=True))


In [ ]:
import requests
import re

def test_spotify_connection_via_proxy():
    url = "https://open.spotify.com/track/11dFghVXANMlKmJXsNCbNl"
    # 代理配置
    local_ip = '100.78.231.77'
    proxies = {
        "http": f"http://{local_ip}:7890",
        "https": f"http://{local_ip}:7890"
    }
    try:
        # 向指定的 URL 发送 GET 请求，通过代理进行

        response = requests.get(url ,proxies=proxies)
#         print(response.text)
#         print(type(response.text))
        
        # 定义正则表达式模式
        pattern = r'<meta property="og:audio" content="(https://[^"]+)"'
        
        # 使用正则表达式查找匹配的内容
        match = re.search(pattern, response.text)
        if match:
            audio_url = match.group(1)
            print(audio_url)
            
        if response.status_code == 200:
            print("Content Sample:", response.text[:200])
        else:
            print("Failed to reach Spotify.")
    except requests.exceptions.RequestException as e:
        print("Error during requests to {0} : {1}".format(url, str(e)))

test_spotify_connection_via_proxy()


In [ ]:
import re

# 定义一些示例数据
data = '''
<meta property="og:audio" content="https://p.scdn.co/mp3-preview/ecc6383aac4b3f4240ae699324573e61c39e6aaf"/>
<meta property="og:audio" content="https://another-example.com/song.mp3"/>
'''

# 正则表达式匹配<meta>标签，其中property为"og:audio"
pattern = r'<meta property="og:audio" content="(https://[^"]+)"'

# 使用re.findall查找所有匹配项
urls = re.findall(pattern, data)

# 输出提取的URL
for url in urls:
    print(url)
